# Options Pricing Dashboard

This notebook provides a comprehensive dashboard for options pricing and risk management analysis using the RMA utilities.

## Setup and Imports

In [ ]:
import sys
import os
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Add parent directory to path to import RMA module
sys.path.append(os.path.dirname(os.path.abspath('')))

# Error handling for imports from RMA.utils
try:
    from RMA.utils import (
        validate_positive,
        validate_probability,
        calculate_log_returns,
        calculate_simple_returns,
        annualize_volatility,
        calculate_historical_volatility,
        time_to_maturity,
        format_percentage,
        calculate_option_payoff,
        calculate_profit_loss,
        calculate_breakeven,
        interpolate_volatility,
        generate_price_paths,
        calculate_sharpe_ratio,
        calculate_max_drawdown
    )
    logger.info("✓ Successfully imported all functions from RMA.utils")
    UTILS_AVAILABLE = True
except ModuleNotFoundError as e:
    logger.error(f"✗ RMA.utils module not found: {e}")
    logger.error("  Please ensure utils.py exists in the RMA directory")
    UTILS_AVAILABLE = False
    print("\n⚠️  WARNING: Could not import RMA.utils module!")
    print("    The dashboard will not function properly.")
    print("    Please check that utils.py exists in the RMA directory.\n")
except ImportError as e:
    logger.error(f"✗ Error importing specific functions from RMA.utils: {e}")
    logger.error("  Some functions may not be available in utils.py")
    UTILS_AVAILABLE = False
    print("\n⚠️  WARNING: Error importing functions from RMA.utils!")
    print(f"    Error details: {e}\n")
except Exception as e:
    logger.error(f"✗ Unexpected error while importing from RMA.utils: {e}")
    UTILS_AVAILABLE = False
    print(f"\n⚠️  WARNING: Unexpected error during import: {e}\n")

# Try importing numpy with error handling
try:
    import numpy as np
    NUMPY_AVAILABLE = True
    logger.info("✓ NumPy imported successfully")
except ImportError as e:
    logger.error(f"✗ NumPy is not available: {e}")
    NUMPY_AVAILABLE = False
    np = None
    print("\n⚠️  WARNING: NumPy not available!")
    print("    Install numpy using: pip install numpy\n")

# Try importing pandas with error handling
try:
    import pandas as pd
    PANDAS_AVAILABLE = True
    logger.info("✓ Pandas imported successfully")
except ImportError as e:
    logger.error(f"✗ Pandas is not available: {e}")
    PANDAS_AVAILABLE = False
    pd = None
    print("\n⚠️  WARNING: Pandas not available!")
    print("    Install pandas using: pip install pandas\n")

# Try importing matplotlib with error handling
try:
    import matplotlib.pyplot as plt
    MATPLOTLIB_AVAILABLE = True
    logger.info("✓ Matplotlib imported successfully")
except ImportError as e:
    logger.error(f"✗ Matplotlib is not available: {e}")
    MATPLOTLIB_AVAILABLE = False
    plt = None
    print("\n⚠️  WARNING: Matplotlib not available!")
    print("    Install matplotlib using: pip install matplotlib\n")

# Try importing seaborn with error handling
try:
    import seaborn as sns
    SEABORN_AVAILABLE = True
    logger.info("✓ Seaborn imported successfully")
except ImportError as e:
    logger.error(f"✗ Seaborn is not available: {e}")
    SEABORN_AVAILABLE = False
    sns = None
    print("\n⚠️  WARNING: Seaborn not available!")
    print("    Install seaborn using: pip install seaborn\n")

# Configure plot style if matplotlib and seaborn are available
if MATPLOTLIB_AVAILABLE and SEABORN_AVAILABLE:
    try:
        plt.style.use('seaborn-v0_8-darkgrid')
        sns.set_palette("husl")
        logger.info("✓ Plot style configured successfully")
    except Exception as e:
        logger.warning(f"⚠ Could not set plot style: {e}")
        print(f"\n⚠️  WARNING: Could not configure plot style: {e}\n")

# Configure pandas display settings if pandas is available
if PANDAS_AVAILABLE:
    try:
        pd.set_option('display.max_columns', None)
        pd.set_option('display.precision', 4)
        logger.info("✓ Pandas display settings configured")
    except Exception as e:
        logger.warning(f"⚠ Could not configure pandas settings: {e}")

# Summary of import status
print("\n" + "="*60)
print("IMPORT STATUS SUMMARY")
print("="*60)
print(f"RMA Utils:    {'✓ Available' if UTILS_AVAILABLE else '✗ NOT AVAILABLE'}")
print(f"NumPy:        {'✓ Available' if NUMPY_AVAILABLE else '✗ NOT AVAILABLE'}")
print(f"Pandas:       {'✓ Available' if PANDAS_AVAILABLE else '✗ NOT AVAILABLE'}")
print(f"Matplotlib:   {'✓ Available' if MATPLOTLIB_AVAILABLE else '✗ NOT AVAILABLE'}")
print(f"Seaborn:      {'✓ Available' if SEABORN_AVAILABLE else '✗ NOT AVAILABLE'}")
print("="*60)

# Check if all critical dependencies are available
if UTILS_AVAILABLE and NUMPY_AVAILABLE and PANDAS_AVAILABLE and MATPLOTLIB_AVAILABLE:
    print("\n✓ All imports successful! Dashboard is ready to use.\n")
else:
    print("\n⚠️  WARNING: Some dependencies are missing!")
    print("   The dashboard may not function properly.")
    print("   Please install missing packages and re-run this cell.\n")

In [ ]:
# Safe wrapper functions for error handling throughout the notebook

def safe_calculate_volatility(prices, **kwargs):
    """
    Safely calculate historical volatility with error handling.
    
    Args:
        prices: Array of prices
        **kwargs: Additional arguments for calculate_historical_volatility
    
    Returns:
        Volatility value or None if calculation fails
    """
    if not UTILS_AVAILABLE:
        print("⚠️  Cannot calculate volatility: utils module not available")
        return None
    
    if not NUMPY_AVAILABLE:
        print("⚠️  Cannot calculate volatility: NumPy not available")
        return None
    
    try:
        volatility = calculate_historical_volatility(prices, **kwargs)
        return volatility
    except ValueError as e:
        print(f"⚠️  ValueError in volatility calculation: {e}")
        return None
    except Exception as e:
        print(f"⚠️  Unexpected error in volatility calculation: {e}")
        return None


def safe_calculate_option_value(spot_prices, strike, option_type="call"):
    """
    Safely calculate option payoff with error handling.
    
    Args:
        spot_prices: Array of spot prices
        strike: Strike price
        option_type: "call" or "put"
    
    Returns:
        Option payoff array or None if calculation fails
    """
    if not UTILS_AVAILABLE:
        print("⚠️  Cannot calculate option payoff: utils module not available")
        return None
    
    try:
        payoff = calculate_option_payoff(spot_prices, strike, option_type)
        return payoff
    except ValueError as e:
        print(f"⚠️  ValueError in option payoff calculation: {e}")
        return None
    except Exception as e:
        print(f"⚠️  Unexpected error in option payoff calculation: {e}")
        return None


def safe_generate_price_paths(S0, mu, sigma, T, steps, paths, seed=None):
    """
    Safely generate price paths with error handling.
    
    Args:
        S0: Initial stock price
        mu: Drift
        sigma: Volatility
        T: Time horizon
        steps: Number of steps
        paths: Number of paths
        seed: Random seed
    
    Returns:
        Price paths array or None if generation fails
    """
    if not UTILS_AVAILABLE:
        print("⚠️  Cannot generate price paths: utils module not available")
        return None
    
    try:
        price_paths = generate_price_paths(S0, mu, sigma, T, steps, paths, seed)
        return price_paths
    except ValueError as e:
        print(f"⚠️  ValueError in price path generation: {e}")
        return None
    except Exception as e:
        print(f"⚠️  Unexpected error in price path generation: {e}")
        return None


def check_dependencies():
    """
    Check if all required dependencies are available.
    
    Returns:
        True if all dependencies are available, False otherwise
    """
    all_available = (UTILS_AVAILABLE and NUMPY_AVAILABLE and 
                    PANDAS_AVAILABLE and MATPLOTLIB_AVAILABLE)
    
    if not all_available:
        print("\n⚠️  WARNING: Not all dependencies are available!")
        if not UTILS_AVAILABLE:
            print("   - RMA.utils is missing")
        if not NUMPY_AVAILABLE:
            print("   - NumPy is missing")
        if not PANDAS_AVAILABLE:
            print("   - Pandas is missing")
        if not MATPLOTLIB_AVAILABLE:
            print("   - Matplotlib is missing")
        print("\nSome cells may fail to execute properly.\n")
        return False
    
    print("✓ All dependencies are available for the dashboard\n")
    return True

# Run dependency check
check_dependencies()

## 1. Historical Volatility Analysis

In [ ]:
# Generate sample price data
np.random.seed(42)
days = 252  # One year of trading days
initial_price = 100
returns = np.random.normal(0.0005, 0.02, days)
prices = initial_price * np.exp(np.cumsum(returns))

# Calculate historical volatility
hist_vol = calculate_historical_volatility(prices, annualize=True)

print(f"Historical Volatility: {format_percentage(hist_vol)}")
print(f"Annualized Volatility: {hist_vol:.4f}")

In [ ]:
# Plot price series
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Price chart
ax1.plot(prices, linewidth=2, color='#2E86AB')
ax1.set_title('Stock Price History', fontsize=14, fontweight='bold')
ax1.set_xlabel('Trading Days')
ax1.set_ylabel('Price ($)')
ax1.grid(True, alpha=0.3)

# Returns distribution
log_returns = calculate_log_returns(prices)
ax2.hist(log_returns, bins=50, alpha=0.7, color='#A23B72', edgecolor='black')
ax2.set_title('Distribution of Log Returns', fontsize=14, fontweight='bold')
ax2.set_xlabel('Log Returns')
ax2.set_ylabel('Frequency')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Rolling Volatility Analysis

In [ ]:
# Calculate rolling volatility with different windows
windows = [20, 50, 100]
rolling_vols = {}

for window in windows:
    rolling_vols[window] = calculate_historical_volatility(prices, window=window, annualize=True)

# Plot rolling volatilities
plt.figure(figsize=(14, 6))
for window, vol in rolling_vols.items():
    plt.plot(range(window, len(prices)), vol, label=f'{window}-day window', linewidth=2)

plt.title('Rolling Historical Volatility', fontsize=14, fontweight='bold')
plt.xlabel('Trading Days')
plt.ylabel('Annualized Volatility')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Option Payoff Analysis

In [ ]:
# Option parameters
current_price = prices[-1]
strike_price = 100
call_premium = 5.0
put_premium = 4.5

# Generate range of spot prices at expiration
spot_range = np.linspace(80, 120, 200)

# Calculate payoffs
call_payoff = calculate_option_payoff(spot_range, strike_price, "call")
put_payoff = calculate_option_payoff(spot_range, strike_price, "put")

# Calculate P&L (long positions)
call_pnl = calculate_profit_loss(call_payoff, call_premium, "long")
put_pnl = calculate_profit_loss(put_payoff, put_premium, "long")

# Calculate breakeven points
call_breakeven = calculate_breakeven(strike_price, call_premium, "call")
put_breakeven = calculate_breakeven(strike_price, put_premium, "put")

print(f"Current Price: ${current_price:.2f}")
print(f"Strike Price: ${strike_price:.2f}")
print(f"\nCall Option:")
print(f"  Premium: ${call_premium:.2f}")
print(f"  Breakeven: ${call_breakeven:.2f}")
print(f"\nPut Option:")
print(f"  Premium: ${put_premium:.2f}")
print(f"  Breakeven: ${put_breakeven:.2f}")

In [ ]:
# Plot payoff diagrams
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Call option
ax1.plot(spot_range, call_payoff, label='Payoff', linewidth=2, color='#2E86AB')
ax1.plot(spot_range, call_pnl, label='P&L (Long)', linewidth=2, color='#A23B72')
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.3)
ax1.axvline(x=strike_price, color='gray', linestyle='--', alpha=0.5, label=f'Strike ${strike_price}')
ax1.axvline(x=call_breakeven, color='green', linestyle='--', alpha=0.5, label=f'Breakeven ${call_breakeven:.2f}')
ax1.set_title('Call Option Payoff Diagram', fontsize=14, fontweight='bold')
ax1.set_xlabel('Spot Price at Expiration ($)')
ax1.set_ylabel('Profit/Loss ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Put option
ax2.plot(spot_range, put_payoff, label='Payoff', linewidth=2, color='#2E86AB')
ax2.plot(spot_range, put_pnl, label='P&L (Long)', linewidth=2, color='#A23B72')
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.3)
ax2.axvline(x=strike_price, color='gray', linestyle='--', alpha=0.5, label=f'Strike ${strike_price}')
ax2.axvline(x=put_breakeven, color='green', linestyle='--', alpha=0.5, label=f'Breakeven ${put_breakeven:.2f}')
ax2.set_title('Put Option Payoff Diagram', fontsize=14, fontweight='bold')
ax2.set_xlabel('Spot Price at Expiration ($)')
ax2.set_ylabel('Profit/Loss ($)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Monte Carlo Simulation

In [ ]:
# Simulation parameters
S0 = current_price
mu = 0.10  # 10% annual drift
sigma = hist_vol  # Use calculated historical volatility
T = 1.0  # 1 year
steps = 252  # Daily steps
num_paths = 1000

# Generate price paths
price_paths = generate_price_paths(S0, mu, sigma, T, steps, num_paths, seed=42)

print(f"Simulation Parameters:")
print(f"  Initial Price: ${S0:.2f}")
print(f"  Drift (μ): {format_percentage(mu)}")
print(f"  Volatility (σ): {format_percentage(sigma)}")
print(f"  Time Horizon: {T} year(s)")
print(f"  Number of Paths: {num_paths:,}")
print(f"\nFinal Price Statistics:")
print(f"  Mean: ${np.mean(price_paths[:, -1]):.2f}")
print(f"  Median: ${np.median(price_paths[:, -1]):.2f}")
print(f"  Std Dev: ${np.std(price_paths[:, -1]):.2f}")
print(f"  Min: ${np.min(price_paths[:, -1]):.2f}")
print(f"  Max: ${np.max(price_paths[:, -1]):.2f}")

In [ ]:
# Plot simulated paths
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Sample paths
sample_paths = min(100, num_paths)
for i in range(sample_paths):
    ax1.plot(price_paths[i], alpha=0.3, linewidth=0.5)
ax1.plot(np.mean(price_paths, axis=0), color='red', linewidth=3, label='Mean Path')
ax1.set_title(f'Monte Carlo Simulation ({sample_paths} Sample Paths)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Time Steps')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Distribution of final prices
ax2.hist(price_paths[:, -1], bins=50, alpha=0.7, color='#A23B72', edgecolor='black')
ax2.axvline(np.mean(price_paths[:, -1]), color='red', linestyle='--', linewidth=2, label='Mean')
ax2.axvline(np.median(price_paths[:, -1]), color='green', linestyle='--', linewidth=2, label='Median')
ax2.set_title('Distribution of Final Prices', fontsize=14, fontweight='bold')
ax2.set_xlabel('Final Price ($)')
ax2.set_ylabel('Frequency')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Option Pricing with Monte Carlo

In [ ]:
# Calculate option prices using Monte Carlo
call_payoffs_mc = calculate_option_payoff(price_paths[:, -1], strike_price, "call")
put_payoffs_mc = calculate_option_payoff(price_paths[:, -1], strike_price, "put")

# Discount to present value (assuming risk-free rate = 0 for simplicity)
call_price_mc = np.mean(call_payoffs_mc)
put_price_mc = np.mean(put_payoffs_mc)

# Calculate confidence intervals
call_ci = 1.96 * np.std(call_payoffs_mc) / np.sqrt(num_paths)
put_ci = 1.96 * np.std(put_payoffs_mc) / np.sqrt(num_paths)

print("Monte Carlo Option Pricing Results:")
print(f"\nCall Option (Strike: ${strike_price}):")
print(f"  Estimated Price: ${call_price_mc:.2f}")
print(f"  95% CI: ±${call_ci:.2f}")
print(f"  In-the-money: {np.sum(call_payoffs_mc > 0) / num_paths * 100:.1f}%")

print(f"\nPut Option (Strike: ${strike_price}):")
print(f"  Estimated Price: ${put_price_mc:.2f}")
print(f"  95% CI: ±${put_ci:.2f}")
print(f"  In-the-money: {np.sum(put_payoffs_mc > 0) / num_paths * 100:.1f}%")

## 6. Risk Metrics

In [ ]:
# Calculate risk metrics for the original price series
log_returns = calculate_log_returns(prices)
sharpe = calculate_sharpe_ratio(log_returns, risk_free_rate=0.02)
max_dd, peak_idx, trough_idx = calculate_max_drawdown(prices)

print("Risk Metrics:")
print(f"\nSharpe Ratio: {sharpe:.4f}")
print(f"\nMaximum Drawdown: {format_percentage(max_dd)}")
print(f"  Peak Index: {peak_idx}")
print(f"  Trough Index: {trough_idx}")
print(f"  Peak Price: ${prices[peak_idx]:.2f}")
print(f"  Trough Price: ${prices[trough_idx]:.2f}")

In [ ]:
# Plot drawdown analysis
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Price with drawdown period highlighted
ax1.plot(prices, linewidth=2, color='#2E86AB', label='Price')
ax1.axvline(x=peak_idx, color='green', linestyle='--', alpha=0.7, label=f'Peak (${prices[peak_idx]:.2f})')
ax1.axvline(x=trough_idx, color='red', linestyle='--', alpha=0.7, label=f'Trough (${prices[trough_idx]:.2f})')
ax1.fill_between(range(peak_idx, trough_idx+1), 
                  prices[peak_idx:trough_idx+1], 
                  prices[peak_idx], 
                  alpha=0.3, color='red', label='Drawdown Period')
ax1.set_title('Price Series with Maximum Drawdown', fontsize=14, fontweight='bold')
ax1.set_xlabel('Trading Days')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Drawdown series
cumulative_max = np.maximum.accumulate(prices)
drawdowns = (prices - cumulative_max) / cumulative_max
ax2.fill_between(range(len(drawdowns)), drawdowns, 0, alpha=0.5, color='red')
ax2.plot(drawdowns, linewidth=1, color='darkred')
ax2.set_title('Drawdown Over Time', fontsize=14, fontweight='bold')
ax2.set_xlabel('Trading Days')
ax2.set_ylabel('Drawdown (%)')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y*100:.0f}%'))
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Implied Volatility Surface (Example)

In [ ]:
# Create sample volatility smile data
strikes = np.array([80, 85, 90, 95, 100, 105, 110, 115, 120])
implied_vols = np.array([0.35, 0.30, 0.25, 0.22, 0.20, 0.22, 0.25, 0.30, 0.35])

# Interpolate for a specific strike
target_strike = 97.5
interpolated_vol = interpolate_volatility(strikes, implied_vols, target_strike)

print(f"Volatility Smile:")
print(f"\nInterpolated volatility at strike ${target_strike}: {format_percentage(interpolated_vol)}")

# Create detailed interpolation for plotting
fine_strikes = np.linspace(strikes.min(), strikes.max(), 100)
fine_vols = np.interp(fine_strikes, strikes, implied_vols)

# Plot volatility smile
plt.figure(figsize=(12, 6))
plt.plot(fine_strikes, fine_vols * 100, linewidth=2, color='#2E86AB', label='Interpolated')
plt.scatter(strikes, implied_vols * 100, s=100, color='#A23B72', zorder=5, label='Market Data')
plt.scatter([target_strike], [interpolated_vol * 100], s=150, color='red', marker='*', 
            zorder=6, label=f'Interpolated @ ${target_strike}')
plt.axvline(x=current_price, color='gray', linestyle='--', alpha=0.5, label=f'Current Price ${current_price:.2f}')
plt.title('Implied Volatility Smile', fontsize=14, fontweight='bold')
plt.xlabel('Strike Price ($)')
plt.ylabel('Implied Volatility (%)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Summary Dashboard

In [ ]:
# Create a summary dataframe
summary_data = {
    'Metric': [
        'Current Price',
        'Historical Volatility',
        'Sharpe Ratio',
        'Maximum Drawdown',
        'Call Price (MC)',
        'Put Price (MC)',
        'Call Breakeven',
        'Put Breakeven'
    ],
    'Value': [
        f'${current_price:.2f}',
        format_percentage(hist_vol),
        f'{sharpe:.4f}',
        format_percentage(max_dd),
        f'${call_price_mc:.2f}',
        f'${put_price_mc:.2f}',
        f'${call_breakeven:.2f}',
        f'${put_breakeven:.2f}'
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*50)
print("DASHBOARD SUMMARY")
print("="*50)
print(summary_df.to_string(index=False))
print("="*50)

## 9. Interactive Analysis Section

Use the cells below to perform custom analysis using the imported utility functions.

In [ ]:
# Your custom analysis here
